In [ ]:
# This cell is only for setting parameter defaults
day_obs = "2024-09-04"
sim_date = "2024-09-04"

# Prenight Briefing for {{ params.day_obs }}

This is presently just a playground for building a pre-night briefing.

In [ ]:
# Validate the inputs
import re
assert re.match(r'^\d\d\d\d-\d\d-\d\d$', day_obs) is not None
assert re.match(r'^\d\d\d\d-\d\d-\d\d$', sim_date) is not None

In [ ]:
from IPython.display import display, HTML
import datetime
import math
import sys
import os
import yaml
from urllib.parse import urlparse
import warnings
import itertools
import pandas as pd
import numpy as np
import astropy
import bokeh
import bokeh.io
import boto3
import colorcet
from erfa import ErfaWarning
from astropy.time import Time

In [ ]:
from sklearn.neighbors import KernelDensity

In [ ]:
os.environ['RUBIN_SIM_DATA_DIR'] = '/sdf/data/rubin/user/neilsen/data/rubin_sim_data'

In [ ]:
sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/pip_targets/lib/python3.11/site-packages')

In [ ]:
devel_versions = True
if devel_versions:
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/uranography')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/rubin_scheduler')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/rubin_sim')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/schedview')

In [ ]:
import rubin_scheduler
import rubin_scheduler.utils
import rubin_scheduler.sim_archive
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_sim import maf
from lsst.resources import ResourcePath

In [ ]:
import schedview.compute
import schedview.compute.visits
import schedview.collect
import schedview.collect.rewards
import schedview.plot
import schedview.plot.rewards

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

# Don't even complain.
warnings.filterwarnings(
    "ignore",
    category=astropy.utils.exceptions.AstropyWarning,
    message="Tried to get polar motions for times after IERS data is valid. Defaulting to polar motion from the 50-yr mean for those. This may affect precision at the arcsec level. Please check your astropy.utils.iers.conf.iers_auto_url and point it to a newer version if necessary.",
)

In [ ]:
# In simulations, we go far enough into the future that the erfa module finds it "dubious".
# Keep the complaints quiet.
warnings.filterwarnings(
    "ignore",
    category=ErfaWarning,
    message=r".*dubious year.*",
)

In [ ]:
# Don't complain about working with daytime MJDs either.
warnings.filterwarnings(
    "ignore",
    module="rubin_scheduler.skybrightness_pre.sky_model_pre",
    category=UserWarning,
    message="Requested MJD between sunrise and sunset, returning closest maps",
)

In [ ]:
warnings.filterwarnings(
    "ignore",
    module=r"schedview.collect.opsim.*",
    category=UserWarning,
    message=r"Column .* not found in .*, skipping.",
)

In [ ]:
# Quiet unimportant chatter from healpy.
healpy_logger = logging.getLogger("healpy")
healpy_logger.setLevel(logging.WARNING)

In [ ]:
bokeh.io.output_notebook()

In [ ]:
%matplotlib inline

In [ ]:
archive_uri = "s3://rubin:rubin-scheduler-prenight/opsim/"

if urlparse(archive_uri).scheme.upper() == 'S3':
    os.environ["LSST_DISABLE_BUCKET_VALIDATION"] = "1"
    os.environ["S3_ENDPOINT_URL"] = "https://s3dfrgw.slac.stanford.edu/"

In [ ]:
day_obs_mjd = int(Time(day_obs).mjd)
observatory = ModelObservatory(init_load_length=1)
timezone = "Chile/Continental"

## Astronomical events during the night

In [ ]:
day_obs_datetime = Time(day_obs_mjd, format='mjd').datetime
day_obs_date = datetime.date(day_obs_datetime.year, day_obs_datetime.month, day_obs_datetime.day)
night_events = schedview.compute.astro.night_events(day_obs_date)
night_events

## Sun and moon positions in the middle of the night

In [ ]:
model_observatory = ModelObservatory(init_load_length=1)
model_observatory.mjd = night_events.loc['night_middle', 'MJD']

In [ ]:
body_positions_wide = pd.DataFrame(model_observatory.almanac.get_sun_moon_positions(night_events.loc['night_middle', 'MJD']))
body_positions_wide.index.name = 'r'
body_positions_wide.reset_index(inplace=True)

angle_columns = ['RA', 'dec', 'alt', 'az']
all_columns = angle_columns + ['phase']
body_positions = (
    pd.wide_to_long(body_positions_wide, stubnames=('sun', 'moon'), suffix=r'.*', sep='_', i='r', j='')
    .droplevel('r')
    .T[all_columns]
)
body_positions[angle_columns] = np.degrees(body_positions[angle_columns])
body_positions

All angles are in degrees.

## Simulated visits

In [ ]:
visits = schedview.collect.read_multiple_opsims(archive_uri, sim_date, day_obs_mjd)
visits_ds = bokeh.models.ColumnDataSource(visits)

In [ ]:
sim_labels = visits['label'].unique()
sim_color_mapper, sim_color_dict, sim_marker_mapper, sim_hatch_dict = schedview.plot.generate_sim_indicators(sim_labels)

## Altitude and airmass

In [ ]:
fig = schedview.plot.multisim.plot_alt_airmass_vs_time(
    visits_ds,
    scatter_user_kwargs=dict(
        fill_alpha=0.2,
        color={"field": "label", "transform": sim_color_mapper},
        marker={"field": "label", "transform": sim_marker_mapper},
    )
)
bokeh.io.show(fig)

## Often repeated fields

An often repeated field is a field repeated at least four times in at least one simulation:

In [ ]:
often_repeated_fields, often_repeated_field_stats = schedview.compute.often_repeated_fields(visits)
often_repeated_field_stats.style.format({
    'first_time': lambda t: t.strftime("%H:%M:%S"),
    'last_time': lambda t: t.strftime("%H:%M:%S")}) 

At present, field coordinates must be exactly matched to be recognized as the "same" field.
A more robust approach would be to find clusters of nearby pointings (maybe with kmeans or a similar algorithm), and group by the identified clusters.

## Distribution comparisons

In [ ]:
fig = schedview.plot.overplot_kernel_density_estimates(visits, column='fieldRA', x_points=np.arange(0, 360), colors=sim_color_dict, hatches=sim_hatch_dict, bandwidth=1)
bokeh.io.show(fig)

In [ ]:
fig = schedview.plot.overplot_kernel_density_estimates(visits, column='fieldRA', x_points=np.arange(0, 360), colors=sim_color_dict, hatches=sim_hatch_dict, bandwidth=1)
bokeh.io.show(fig)

In [ ]:
fig = schedview.plot.overplot_kernel_density_estimates(visits, column='fieldDec', x_points=np.arange(-90, 30), colors=sim_color_dict, hatches=sim_hatch_dict, bandwidth=1)
bokeh.io.show(fig)

In [ ]:
fig = schedview.plot.overplot_kernel_density_estimates(visits, column='airmass', bandwidth=0.001, x_points=np.arange(1.0, 2.5, 0.005), colors=sim_color_dict, hatches=sim_hatch_dict)
bokeh.io.show(fig)

## Common visits

In [ ]:
visit_counts = (
    visits
    .groupby(['sim_index', 'fieldRA', 'fieldDec', 'filter', 'visitExposureTime'])
    .agg({'start_date': 'count'})
    .rename(columns={'start_date': 'count'})
    .reset_index()
    .pivot(index=['fieldRA', 'fieldDec', 'filter', 'visitExposureTime'],
           columns=['sim_index'],
           values='count')
    .fillna(0)
    .astype(int)
)

Coordinate/filter/exposure time combinations repeated more that four times in any simulation:

In [ ]:
visit_counts.loc[visit_counts.max(axis='columns')>4, :]

Coordinate/filter/exposure time combinations all simulations have in common, and the count from the simulation where it is repeated the fewest times:

In [ ]:
common_visits = visit_counts.min(axis='columns')
common_visits = common_visits[common_visits>0]
print(f"There are {common_visits.sum()} visits in common across all simulations:")
common_visits

In [ ]:
def fraction_common(visit_counts, sim1, sim2, match_count=True):
    # Only count fields for which there is at least one visit in sim1
    these_visit_counts = visit_counts[visit_counts[sim2] > 0]

    if not match_count:
        # Count any number of repetitons of a field only once
        for sim in sim1, sim2:
            these_visit_counts.loc[these_visit_counts[sim]>0, sim] = 1

    num_common_visits = these_visit_counts[[sim2, sim1]].min(axis='columns').sum()
    num_visits2 = these_visit_counts[sim2].sum()
    fraction_common = num_common_visits/num_visits2
    return fraction_common


In [ ]:
def make_fraction_common_matrix(visit_counts, match_count=True, sim_indexes=None):
    if sim_indexes is None:
        sim_indexes = visit_counts.columns.values

    common_matrix = pd.DataFrame(np.nan, index=sim_indexes, columns=sim_indexes)
    for row in sim_indexes:
        for column in sim_indexes:
            common_matrix.loc[row, column] = fraction_common(visit_counts, row, column, match_count=match_count)
    return common_matrix

Matrix of fraction of coordinate/filter/exposure time combinations present in one simulation that are alse present in another.

For example, column 1, row 2 has the fraction of such combinations present in simulation one that are also present is simulation 2.

In [ ]:
make_fraction_common_matrix(visit_counts, match_count=False)

Matrix of fraction of coordinate/filter/exposure time combinations present in one simulation that are alse present in another, where repeats in both are considered additional matches and differences in number of repeats of a given combination are counted as occurrences in one but not the other.

In [ ]:
make_fraction_common_matrix(visit_counts)    

## Timing offsets

In [ ]:
def find_best_matches(start_times, sim_indexes=(1, 2), max_match_dist=np.inf):
    for sim_index in sim_indexes:
        if sim_index not in start_times.index:
            return pd.DataFrame({sim_indexes[0]: [], sim_indexes[1]: [], 'delta': []})
    
    if len(start_times.loc[[sim_indexes[0]]]) >= len(start_times.loc[[sim_indexes[1]]]):
        sim_map = {'longer': sim_indexes[0], 'shorter': sim_indexes[1]}
    else:
        sim_map = {'longer': sim_indexes[1], 'shorter': sim_indexes[0]}

    longer = start_times.loc[[sim_map['longer']]]
    shorter = start_times.loc[[sim_map['shorter']]]

    num_combinations = math.comb(len(longer), len(shorter))
    if num_combinations > 1000:
        # There are too many combinations to do a complete search in reasonable time
        # so assume the matches are sequential.
        def make_seq_iter(num_offsets, sequence_length):
            for i in range(num_offsets):
                yield np.arange(sequence_length) + i
        combo_iterator = make_seq_iter(len(longer)-len(shorter), len(shorter))
    else:
        combo_iterator = itertools.combinations(np.arange(len(longer)), len(shorter))
    
    matches = pd.DataFrame({'longer': longer.values[:len(shorter)], 'shorter': shorter.values})
    best_diff = np.inf
    most_matches = 0
    for matched_ids in combo_iterator:
        matches.loc[:, 'longer'] = longer.values[np.array(matched_ids)]
        matches['delta'] = (matches.shorter - matches.longer).dt.total_seconds()
        good_matches = matches.query(f'abs(delta) < {max_match_dist}')
        num_matches = len(good_matches)
        max_diff = matches['delta'].max()
        within_max_dist = max_diff <= max_match_dist
        if (num_matches >= most_matches) and (max_diff < best_diff):
            best_match = good_matches.copy()
            best_diff = max_diff
            most_matches = num_matches

    best_match.rename(columns=sim_map, inplace=True)
    return best_match

In [ ]:
def compute_best_match_delta_stats(these_visits, sim_indexes):
    return find_best_matches(these_visits.set_index('sim_index').start_date, sim_indexes).loc[:,'delta'].describe()

In [ ]:
reference_index = 1
delta_stats_list = []
for comparison_index in visits['sim_index'].unique():
    if comparison_index == reference_index:
        continue
    sim_indexes = [reference_index, comparison_index]
    these_delta_stats = visits.groupby(['fieldRA', 'fieldDec', 'filter', 'visitExposureTime']).apply(compute_best_match_delta_stats, sim_indexes=sim_indexes).query('count > 0')
    these_delta_stats['sim_index'] = comparison_index
    delta_stats_list.append(these_delta_stats)

pd.concat(delta_stats_list).set_index('sim_index', append=True).sort_index()